
### Notebook

This is a notebook to check out the model. The notebook allows you to visualize individual scans, the identified objects and the estimated surfaces for the scans.


In [1]:
import open3d as o3d
from scripts.preproc_functions import preproc, cluster_points
from scripts.volume_functions import compute_poisson_volume
from scripts.helper_functions import load_hyperparam_search_results
from scripts.helper_functions import material_settings

import itertools
from itertools import product
import numpy as np

sample = "sample7"
material = "træ"

data = o3d.io.read_triangle_mesh(f"Data/{material}/{sample}/textured_output.obj")
object_points = preproc(data,
                        segment     = True,
                        threshold   = 0.02,
                        crop        = False,
                        center      = True,
                        n_points    = 100000)

# Visualize only the preprocessed points
#o3d.visualization.draw_geometries([object_points])

# Visualize only the original data
#o3d.visualization.draw_geometries([data])

# Visualize the original data with the preprocessed pooints and a coordinate frame. 
#coordinate_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.5, origin=[0, 0, 0])
#o3d.visualization.draw_geometries([data,object_points,coordinate_frame])

In [2]:
# Two-step cluster the preprocessed points.
object_cluster = cluster_points(object_points,  choice=4, eps=0.1, min_points=10)
object_cluster = cluster_points(object_cluster, choice=3, eps=0.03, min_points=10)

# Visualize the clustered points    
#o3d.visualization.draw_geometries([object_cluster])    

In [3]:
# Fetch the optimal parameters for a given material
overestimation, ldt, true_volume = material_settings(material)

# Compute volume
volume, mesh = compute_poisson_volume(
    object_cluster,
    true_volume           = true_volume,
    depth                 = 10,
    low_density_threshold = ldt,
    cut_mesh              = True,
    close_mesh            = False,
    close_mesh_adv        = True,
    ground_threshold      = 1,
    ground_level          = 'auto',
    edge_threshold        = 1,
    overestimation        = overestimation, 
    print_flag            = True
                          )


# Visualize the points and the mesh
#o3d.visualization.draw_geometries([object_cluster,mesh],point_show_normal=True,mesh_show_back_face=False)

#o3d.visualization.draw_geometries([mesh],point_show_normal=False)



True Volume       5.15508
Estimated volume: 5.32803 Liters
Absolute error:   0.17 Liters
Percentage error: 3.35 %


# Compute volume for a list of samples

In [4]:
from scripts.helper_functions import samples_run
# Run over test data
# Define list of materials
materials = ['iso','mur','gips','træ','es']

# Define the samples
samples     = ["sample5",
               "sample6", 
               "sample7", 
               "sample8"]

pred_materials,errors_materials = samples_run(samples,materials)

Running model on material iso (...)
Running model on material mur (...)
Running model on material gips (...)
Running model on material træ (...)
Running model on material es (...)
Done!


In [5]:
from scripts.helper_functions import sample_results
# Print result statistics for a given material

# Set the material ID to print results for.
ID = 0
material = materials[ID]
sample_results(material,pred_materials,errors_materials,ID)

Material: iso
True Volume: 93.06 liters

Predictions: [90.57564954662085, 86.34301233768537, 92.48975886805128, 90.32803301756365]
Mean Prediction: 89.9
Std Error (Prediction): 1.1

Absolute Errors:
Mean: 3.1
Std Error: 1.1
Mean Percent Error: 3.4%
Std Percent Error: 1.2%

Mean Errors:
Mean: -3.1
Std Error: 1.1
Mean Percent Error: 3.4%
Std Percent Error: 1.2%


# Hyperparam results

Optionally load and inspect results from hyperparameter optimization.

In [9]:
# If using saved hyperparam results:
result_file = "results/hyperparam_results.json"
material_params = load_hyperparam_search_results(result_file,print_results=False)
material = 'mur'
params = dict(material_params[material])
print(params)

# Unpack parameters
#segment_threshold = params['segment_threshold']
#crop_length       = params['crop_length']
#choice1           = params['choice1']
#eps1              = params['eps1']
#min_points1       = params['min_points1']
#choice2           = params['choice2']
#eps2              = params['eps2']
#min_points2       = params['min_points2']
#depth             = params["depth"]
#ldt               = params['ldt']
#cut_mesh          = params["cut_mesh"]
#close_mesh        = params["close_mesh"]
#ground_threshold  = params['gt']
#overestimation    = params['overestimation']

{'choice1': 4, 'choice2': 3, 'close_mesh': False, 'close_mesh_adv': True, 'cut_mesh': True, 'depth': 10, 'edge_threshold': 1, 'eps1': 0.1, 'eps2': 0.03, 'ground_level': 'auto', 'gt': 1, 'ldt': 0.1, 'min_points1': 10, 'min_points2': 10, 'overestimation': 0.88, 'segment_threshold': 0.02}
